### OBJECT RECOGNITION IN IMAGES USING CIFAR DATASET

In [ ]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import tflearn
from tflearn.layers.core import fully_connected, dropout, input_data
from tflearn.layers.estimator import regression
from tflearn.layers.conv import conv_2d, max_pool_2d
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tflearn.layers.normalization import local_response_normalization
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### load the data
CIFAR-10 Dataset<br>
Credits: Alex Krizhevsky https://www.cs.toronto.edu/~kriz/cifar.html.


In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
dic_1 = unpickle('cifar-10-batches-py/data_batch_1')
dic_2 = unpickle('cifar-10-batches-py/data_batch_2')
dic_3 = unpickle('cifar-10-batches-py/data_batch_3')
dic_4 = unpickle('cifar-10-batches-py/data_batch_4')
dic_5 = unpickle('cifar-10-batches-py/data_batch_5')

In [ ]:
data = np.concatenate((dic_1[b'data'], dic_2[b'data'],dic_3[b'data'], dic_4[b'data'], dic_5[b'data']), axis=0)

In [ ]:
labels = np.concatenate((dic_1[b'labels'], dic_2[b'labels'],dic_3[b'labels'], dic_4[b'labels'], dic_5[b'labels']), axis=0)

In [ ]:
label_names = unpickle('cifar-10-batches-py/batches.meta')[b'label_names']

In [ ]:
data.shape

### splitting data into training and testing using sklearn

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size = 0.2)

### data preprocessing 

various method for data preprocessing, use either one of them<br>
$Method$  $1$:<br> 
&nbsp;&nbsp;&nbsp;&nbsp;1) First of all using dstack for stacking arrays in sequence depth wise as given dataset is in rgb format i.e. converting 3072 into three parts i.e. 1024 each for r,g and b colors, and normalizing data, by dividing by 255 as each bit ranges from 0-255 or can use standardscaler<br> 
$Method$ $2$:<br> 
&nbsp;&nbsp;&nbsp;&nbsp;Similar to method one just after that we can convert rgb to just grayscale data, thus reducing shape from (?, 32, 32, 3) to (?, 32, 32), thus increasing overall speed<br> 
$Method$ $3$:<br> 
&nbsp;&nbsp;&nbsp;&nbsp;Using PCA to speed up:<br>
&nbsp;&nbsp;&nbsp;&nbsp;1) First we need to standardize the data using standard scalar, i.e. with mean of zero and a standard deviation of one.<br>
&nbsp;&nbsp;&nbsp;&nbsp;2) After standardization we pass it to PCA, with a variance of 0.95 which reduces 3072 features to just 221 whoes variance is greater than or equal to 0.95<br>


#### Method 1

In [ ]:
scalar = StandardScaler()
train_x = train_x.astype(float)
train_x = scalar.fit_transform(train_x)
test_x = test_x.astype(float)
test_x = scalar.transform(test_x)

In [ ]:
train_x = np.dstack((train_x[:, :1024], train_x[:, 1024:2048], train_x[:, 2048:]))
test_x = np.dstack((test_x[:, :1024], test_x[:, 1024:2048], test_x[:, 2048:]))

In [ ]:
train_x = np.reshape(train_x, [-1, 32, 32, 3])
test_x = np.reshape(test_x, [-1, 32, 32, 3])

In [ ]:
train_x.shape

In [ ]:
tr_x = train_x

#### Method 2

In [ ]:
train_x = np.dstack((train_x[:, :1024], train_x[:, 1024:2048], train_x[:, 2048:])) / 255
test_x = np.dstack((test_x[:, :1024], test_x[:, 1024:2048], test_x[:, 2048:])) / 255

In [ ]:
train_x = np.reshape(train_x, [-1, 32, 32, 3])
test_x = np.reshape(test_x, [-1, 32, 32, 3])

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

gray = rgb2gray(train_x)
gray_test = rgb2gray(test_x)
gray.shape

In [ ]:
f, a = plt.subplots(2, 10, figsize=(10, 3))

for i in range(10):
    
    a[0][i].imshow(gray[i])
    a[1][i].imshow(train_x[i])

In [ ]:
train_x.shape

In [ ]:
gray.shape

In [ ]:
tr_x = gray
test_x = gray_test

#### Method 3

In [ ]:
scalar = StandardScaler()
train_x = train_x.astype(float)
train_x = scalar.fit_transform(train_x)
test_x = test_x.astype(float)
test_x = scalar.transform(test_x)

In [ ]:
pca = PCA(.95)

In [ ]:
pca.fit(train_x)

In [ ]:
tr_x = pca.transform(train_x)

In [ ]:
tr_x.shape

#### converting output number to one hot vector

In [ ]:
y_train_onehot = pd.get_dummies(train_y).values

### Model building

Can try various options for optimization of hyperparameters for better accuracy<br>
1) learning rate<br>
2) batch_size<br>
3) n_epoch<br>
4) one layer cnn<br>
5) multiple layer cnn<br>
6) dropout<br>
7) image augmentation<br>
8) regularization

input shape needs to be adjusted based upon preprocessing method

In [ ]:
tf.reset_default_graph()
fnn = input_data(shape=[None, 32, 32, 3])
#fnn = tf.reshape(fnn,[-1,221,1])
#fnn = tf.reshape(fnn, [-1, 16, 16, 4])
#print (fnn.shape)
fnn = conv_2d(fnn, 32, 4, activation = 'relu', regularizer='L2')
fnn = max_pool_2d(fnn, 2)
#fnn = local_response_normalization(fnn)
#fnn = conv_2d(fnn, 64, 4, activation = 'relu', regularizer='L2')
#fnn = max_pool_2d(fnn, 2)
#fnn = fully_connected(fnn,442, activation='relu')
fnn = dropout(fnn, 0.1)
fnn = fully_connected(fnn,256, activation='relu')
fnn = fully_connected(fnn, 10, activation='softmax')
fnn = regression(fnn, loss='categorical_crossentropy', learning_rate = 0.001, optimizer = 'adam')
model = tflearn.DNN(fnn)
model.fit(np.asarray(tr_x), np.asarray(y_train_onehot), n_epoch=100, batch_size=100, show_metric=True)


### Measuring model performance

In [ ]:
predict = model.predict(test_x)
pre = tf.Session().run(tf.argmax(predict,1))
np.sum(pre==test_y)/len(test_y)